In [413]:
from pdfextractor import extractor
import pdfextractor
import pickle
import jieba
import pandas as pd
import re
import requests
import operator

In [451]:
class jmdPDFMap(object):
    """
    根据pdf生成文章，并计算推荐的准确率和召回率
    意图：导语，业绩变动原因，主营业务$业务进展，未来计划
    """
    def __init__(self):
        self.labels = ['导语', '业绩变动原因', '主营业务$业务进展', '未来计划', 'other']
        self.labelmap = {
                         '导语': ['意图_企业业绩_现状偏好', '意图_企业业绩_现状偏坏', '导语', '意图_企业基本信息介绍_企业基本信息介绍', '意图_企业活动_公司公告信息'],
                         '业绩变动原因': ['意图_企业业绩_业绩变动原因'],
                         '主营业务$业务进展': ['意图_企业业务_业务介绍', '意图_企业业务_业务规模', '意图_企业业务_业务规划', '意图_企业活动_人事活动', '意图_企业活动_融资活动','意图_企业活动_投资活动'],
                         '未来计划': ['意图_企业业务_业务前景', '意图_企业业绩_前景偏好','意图_企业业绩_前景偏好'],
                         'other': ['其他', '意图_评论_风险提示','意图_评论_投资建议', '意图_评论_盈利预测']
                        }
        self.labelmapR = {
                          '意图_企业业绩_现状偏好': '导语', '意图_企业业绩_现状偏坏': '导语', '导语': '导语', '意图_企业基本信息介绍_企业基本信息介绍': '导语', 
                          '意图_企业业绩_业绩变动原因': '业绩变动原因',
                          '意图_企业业务_业务介绍': '主营业务$业务进展', '意图_企业业务_业务规模': '主营业务$业务进展', '意图_企业业务_业务规划': '主营业务$业务进展', 
                          '意图_企业业务_业务前景': '未来计划', '意图_企业业绩_前景偏好': '未来计划', '意图_企业业绩_前景偏坏': '未来计划',
                          '意图_企业活动_公司公告信息': '主营业务$业务进展', '意图_企业活动_投资活动': '主营业务$业务进展', '意图_企业活动_人事活动': '主营业务$业务进展', '意图_企业活动_融资活动': '主营业务$业务进展', '其他': 'other', '意图_评论_风险提示': 'other', '意图_评论_投资建议': 'other', '意图_评论_盈利预测':'other'
                         }
        self.prop = [0.2, 0.2, 0.4, 0.3, 0]
        self.K = 0
        #每个意图对应的句子或段落数量
        self.num = {}
        self.recommendList = []
        
        
    def computePR(self, article, textList = None, pdfurl = None, recList = None, mode = 'para'):
        """
        pdfurl和textList为内容来源，必须有且仅有一个为None
        mode 必须指明
        """
        intentList = []
        #article_sen = self.get_article_sentence(article)
        article_sen = article
        article_len = len(article_sen)
        if article_len == 0:
            return -1, -1
        print 'article length: ' + str(article_len)
        if recList is None:
            if not pdfurl is None:
                if mode == 'para':
                    try:
                        pdftext = self.extractPDF(pdfurl, 0.8, ['para'])
                    except BaseException:
                        return -1, -1
                else:
                    try:
                        pdftext = self.extractPDF(pdfurl, 0.8, ['text'])
                    except BaseException:
                        return -1, -1
            else:
                pdftext = textList
            for t in pdftext:
                intentList.append(self.getIntent(t))
            if mode == 'para':
                self.K = int(len(article_sen) * 0.5) + 1
            else:
                self.K = int(len(article_sen) * 1.5) + 1
            for l in range(len(self.labels)):
                if l == len(self.labels) - 1:
                    self.num[self.labels[l]] = 0
                else:
                    self.num[self.labels[l]] = int(self.K * self.prop[l]) + 1
            self.recommendList = self.getTopKintent(intentList)
        else:
            self.recommendList = recList
        rec_len = len(self.recommendList)
        if rec_len == 0:
            return -1, -1
        print 'recommend list length: ' + str(rec_len)
        P_dic = {}
        Pcount = 0
        Rcount = 0
        for r in self.recommendList:
            P_dic[r] = False
        for sen in article_sen:
            for rec in self.recommendList:
                if self.MatchTwoText(sen, rec, 0.6):
                    Rcount += 1
                    if P_dic[rec] is False:
                        P_dic[rec] = True
                        Pcount += 1
                    break
        P = Pcount * 1.0 / rec_len
        R = Rcount * 1.0 / article_len
        print 'Pcount: ' + str(Pcount)
        print 'Rcount: ' + str(Rcount)
        return P, R
    
    def getRec(self):
        return self.recommendList
        
    def getTopKintent(self, intentlist):
        intent = {}
        for l in self.labels:
            intent[l] = {}
        
        for i in intentlist:
            key = i['intent']
            intent[key][i['content']] = i['prob']
        result = []
        for l in self.labels:
            intent_l = intent[l]
            intent[l] = sorted(intent_l.items(), key=operator.itemgetter(1))
        print "各个段落（句子）的预测意图分布为："
        for i in intent:
            print i + "  " + str(len(intent[i]))
        for l in intent.keys():
            if self.num[l] == 0:
                continue
            result.extend([d[0] for d in intent[l][-self.num[l]:]])
        return result
        
    def get_article_sentence(self, ar):
        """
        given an article, return the list of sentences
        """
        textList = ar.encode('utf-8').split('。') 
        textList = [t.replace('\n', '') for t in textList]
        textList = filter(lambda x: x != '', textList)
        return textList

    #percent: 取pdf的前多少页
    def extractPDF(self, pdfPath, percent, source = ['table', 'text', 'para']):
        savePath = '/home/zhuyuhe/mydata/jmd/percent/'
        ex = extractor.StockFinanceExtractor(pdfPath, percent)

        nodes = ex.get_pdf_objects(filter_headline=True)

        paraList = []
        textList = []
        textNodes = []
        tableList = []
        tableNodes = []
        senList = []
        for node in nodes:
            if isinstance(node.obj, pdfextractor.pdftree.VirtualNode):
                continue
            if isinstance(node.obj, pdfextractor.page.Table):
                tableNodes.append(node)
            elif  not isinstance(node.obj, pdfextractor.page.Catalog):
                textNodes.append(node)
        if 'table' in source:
            tableList = self.get_node_texts(tableNodes)
            tableList = [t.replace('|', '').split('\n') for t in tableList]
            #二维数组转化为一维
            tableList = sum(tableList,[])
        textList = self.get_node_texts(textNodes)
        if 'para' in source:
            paraList = [t.replace('\n', '') for t in textList]
            paraList = filter(lambda x: x != '', paraList)
        elif 'text' in source:
            senList = sum([t.encode('utf-8').split('。') for t in textList],[])
            senList = [t.replace('\n', '') for t in senList]
            senList = filter(lambda x: x != '', senList)

        pdf = []
    #     pdf.append(textList)
    #     pdf.append(tableList)
    #     pdf = sum(pdf, [])
#         print pdf
        pdf.extend(senList)
#         print 'after extent senList: ' + str(pdf) 
        pdf.extend(tableList)
#         print 'after extent tableList: ' + str(pdf) 
        pdf.extend(paraList)
#         print 'after extent paraList: ' + str(pdf) 
        print 'pdflist length: ' + str(len(pdf))

        return pdf
    
    def get_node_texts(self, pdf_nodes):
        '''
        Given a list of nodes, get the texts of these nodes
        '''
        texts = []
        for node in pdf_nodes:
            text = node.obj.get_text()
            if len(text) <= 25:
                continue
            if u'□' in text or u'√' in text:
                continue
            texts.append(text)
        return texts
    
    def getIntent(self, text):
        """
        return intent of the given text
        result: {'content': __, 'intent': __ , 'prob': __}
        """
        result = {}
        query = requests.post("https://surreal.aidigger.com/api/v1/du/finance",json=text).json()
        predict = query['predict']
        
        result['content'] = text
        result['intent'], result['prob'] = self.intentMap(predict)
        return result
    
    def intentMap(self, predict):
        """
        将现有意图映射到芥末堆所给的意图上去, 并将权重求和
        """
        result = {}
        for l in self.labels:
            result[l] = 0
#         for p in predict:
#             result[self.labelmapR[p['name'].encode('utf-8')]] += p['prob']
#         maxP = max(result.values())
#         intent = result.keys()[result.values().index(maxP)]
#         return intent, maxP
        for key in predict.keys():
            result[self.labelmapR[key.encode('utf-8')]] += predict[key]
        return result
        
        
    def MatchTwoText(self, t1, t2, threshold = 0.5):
        """
        检查两个句子是否匹配
        """
        ct1 = ' '.join(jieba.cut(t1)).split()
        ct2 = ' '.join(jieba.cut(t2)).split()
        commonL = set(ct1).intersection(set(ct2)).__len__()
        if commonL * 1.0 / set(ct1).__len__() > threshold:
            return True
        return False

    def splitByPeriod(self, P):
        """
        split paragraph to sentences
        """
        senList = sum([t.encode('utf-8').split('。') for t in P],[])
        senList = [t.replace('\n', '') for t in senList]
        senList = filter(lambda x: x != '', senList)
        return senList

a = jmdPDFMap()

In [456]:
k = jmdata_map.keys()[0]
rec = []
for v in jmdata_map[k]['recpara'].values():
    rec.extend(v)
rec = list(set(rec))
for r in rec:
    print r

江苏秀强玻璃工艺股份有限公司是一家主营玻璃深加工的建筑、建材企业.公司主要产品包括家电玻璃、太阳能玻璃、建筑玻璃等。除上述主营业务之外，秀强股份还有教育主营业务。
1、收到的税费返还较上年同期增长839.03%，主要是收到的出口退税增加所致；2、购买商品、接受劳务支付的现金较上年增长43.45%，主要是报告期内公司应付票据到期付款所致；3、支付的各项税费较去年增长74.72%，主要是应缴增值税增加所致；4、取得投资收益收到的现金较去年增长4158.15%，主要是报告期内募集资金购买理财产品产生的收益增加所致；
威创集团股份有限公司是一家科技企业,公司的主营业务是VW系列产品和IDB的研发、生产、销售和服务,为客户提供可视化信息沟通的整体解决方案,公司主要产品为VW和IDB。目前有超高分辨率数字拼接墙系统业务（VW业务）和幼儿园运营管理服务业务两项主营业务。
报告期内，公司实现营业总收入1,602,616,921.65元，较上年1,418,532,229.21元增长12.98%；营业利润-36,587,603.03元，较上年135,276,722.15元下降127.05%；归属于上市公司股东净利润为-62,744,686.37元，较上年100,865,663.75元下降162.21%。
报告期内，公司实现营业总收入1,602,616,921.65元，较上年1,418,532,229.21元增长12.98%；营业利润-36,587,603.05元，较上年135,276,722.15元下降127.05%；归属于上市公司股东净利润为-62,744,686.37元，较上年100,865,663.75元下降162.21%。报告期内，公司内生式增长与外延式增长并重。内生增长方面，公司继续推进LED领域的发展，在通用照明与移动照明领域继续深入开展LED应用产品研发、生产和销售；外延增长方面，积极向教育产业延伸拓展，为公司业绩寻找新的增长点，促进公司可持续发展。
教育消费逐步升级。随着中国普通家庭资产的不断增长、教育观念的发展，家庭对教育也将提出更高的要求，将逐步转向高端消费，从而促使教育机构业务不断向细分领域发展，教育的内在内容与体验将称为教育业务的核心竞争力。教育行业发展不均衡。教育从主体学历教育到配套教育服务，细分领域众多，品牌林立，市场集中度相对较低，多数细分领域属于完全竞争

In [443]:
def getPredict(mode = 'para'):
    for k in jmdata_map.keys():
        text = []
        for p in jmdata_map[k][mode]:
            text.append({'content': p})
        intent = requests.post("https://surreal.aidigger.com/api/v1/du/finance",json=text).json()
        intent = {x['content']: x['features']['intents'] for x in intent}
        for key, value in intent.items():
            intent[key] = {x['name']: x['prob'] for x in value}
            intent[key] = a.intentMap(intent[key])
        jmdata_map[k][mode+'Intent'] = intent
        
def getRec(mode = 'para'):
    num = {}
    labels = ['导语', '业绩变动原因', '主营业务$业务进展', '未来计划', 'other']
    prob = [0.2, 0.3,0.4,0.2,0]
    K = 0
    
    for pdfurl in jmdata_map.keys():
        if mode == 'para':
            K = 0.5 * len(jmdata_map[pdfurl]['content'])
        else:
            K = 1.5 * len(jmdata_map[pdfurl]['content'])
        for i in range(len(labels)):
            if i == 4:
                num[labels[i]] = 0
                continue
            num[labels[i]] = (int)(K * prob[i]) + 1
        recList = {}
        for i in labels[:-1]:
            recList[i] = []
        intent = {}
        for l in labels:
            intent[l] = {}
        for k, v in jmdata_map[pdfurl][mode+'Intent'].items():
            for i, p in v.items():
                intent[i][k] = p
        for i in intent.keys():
            intent[i] = sorted(intent[i].items(), key=operator.itemgetter(1))
            if num[i] == 0:
                continue
            recList[i] = [l[0] for l in intent[i][-num[i]:]]
        jmdata_map[pdfurl]['rec'+mode] = recList

jmdata_map = pickle.load(open('/home/zhuyuhe/mydata/jmd/percent/jmd_pdf_map_headline.pkl', 'rb'))['year']
url = []
for p in jmdata_map.keys():
    if jmdata_map[p]['mapcontent'].__len__() <= 5:
        url.append(p)
for u in url:
    del jmdata_map[u]
#换了api，使用学优要求的推荐方法，先不确定每个句子的意图，每个意图推荐出前n个，然后取list(set())
getPredict('para')
getPredict('sen')

getRec('para')
getRec('sen')

In [442]:
jmdata_map_headline = {}
jmdata_map_headline['season'] = jmdata_map

In [444]:
#增加not_match_content_para, not_match_pdf_para,not_match_content_sen, not_match_pdf_sen
#not_match_content_para: 没有和推荐段落匹配上的文章句子
def MatchTwoText(t1, t2, pdflist,rmlist, threshold = 0.5):
        """
        检查两个句子是否匹配
        """
        ct1 = ' '.join(jieba.cut(t1)).split()
        ct2 = ' '.join(jieba.cut(t2)).split()
        commonL = set(ct1).intersection(set(ct2)).__len__()
        if commonL * 1.0 / set(ct1).__len__() > threshold:
            if not t2 in rmlist:
                rmlist.append(t2)
            return True
        return False

def getNotMatch(jmdata_map, key, mode):
    articleList = list(jmdata_map[key]['mapcontent'])
    rec = []
    for v in jmdata_map[key]['rec'+mode].values():
        rec.extend(v)
    rec = list(set(rec))
    print len(articleList), len(rec)
    rml = []
    rmr = []
    for ar in articleList:
        for text in rec:
            if MatchTwoText(ar, text, rec,rml, threshold=0.6):
                rmr.append(ar)
                break
    for r in rmr:
        articleList.remove(r)
    jmdata_map[key]['not_match_content_' + mode] = articleList
    not_match_pdf = {}
    modeIntent = jmdata_map[key][mode+'Intent']
    
    for r in rml:
        rec.remove(r)
    for r in rec:
        d = modeIntent[r]
        intent = d.keys()[d.values().index(max(d.values()))]
        not_match_pdf[r] = intent
    jmdata_map[key]['not_match_pdf_' + mode] = not_match_pdf
    
#jmdata_map = pickle.load(open('/home/zhuyuhe/mydata/jmd/percent/jmdata_map.pkl', 'rb'))
for key in jmdata_map.keys():
    getNotMatch(jmdata_map, key, 'para')
    getNotMatch(jmdata_map, key, 'sen')

10 8
10 21
9 8
9 21
13 9
13 28
22 17
22 47
8 9
8 21
6 9
6 21
8 10
8 24
15 11
15 30
6 6
6 18
9 9
9 21
8 11
8 28
12 10
12 31
7 9
7 22
6 6
6 13
16 15
16 36
6 8
6 19
9 9
9 22
20 15
20 42
7 12
7 27


In [348]:
i = 0
import sys
stdout = sys.stdout
reload(sys)
sys.setdefaultencoding('utf-8')
sys.stdout = stdout

for key in jmdata_map.keys():
    fileObject = open('/home/zhuyuhe/mydata/jmd/rec/article' + str(i), 'w')
    fileObject.write('0. pdfurl:' + '\n')
    fileObject.write('---------------------' + '\n')
    fileObject.write(key)
    fileObject.write('\n\n\n' + '1. 文章内容为：' + '\n')
    fileObject.write('---------------------' + '\n')
    for s in jmdata_map[key]['content']:
        fileObject.write(s.encode('utf-8')+'\n')
        fileObject.write('-'+'\n')
    fileObject.write('\n\n\n' + '2. 来自pdf的文章内容为:'+'\n')
    fileObject.write('----------------------' + '\n')
    for s in jmdata_map[key]['mapcontent']:
        fileObject.write(s.encode('utf-8')+'\n')
        fileObject.write('-'+'\n')
    fileObject.write('\n\n\n' + '3. 推荐的pdf段落为:' + '\n')
    fileObject.write('-----------------------' + '\n')
    rp = jmdata_map[key]['recpara']
    for k in rp.keys():
        fileObject.write('---' + k + '--- '+ '\n')
        for r in rp[k]:
            fileObject.write(r + '\n')
            fileObject.write('-' + '\n')
        fileObject.write('---' + '\n')
    fileObject.write('\n\n\n' + '4. 没有和推荐段落匹配上的文章句子为：' + '\n')
    fileObject.write('------------------------' + '\n')
    for s in jmdata_map[key]['not_match_content_para']:
        fileObject.write(s.encode('utf-8') + '\n')
        fileObject.write('-' + '\n')
    
    fileObject.write('\n\n\n' + '5. 没有和文章句子匹配上的推荐段落及其意图为：' + '\n')
    fileObject.write('------------------------' + '\n')
    for k, v in jmdata_map[key]['not_match_pdf_para'].items():
        fileObject.write(k + '\t' + v + '\n')
        fileObject.write('-' + '\n')
    
    fileObject.write('\n\n\n' + '6. 推荐的pdf句子为:' + '\n')
    fileObject.write('-----------------------' + '\n')
    rs = jmdata_map[key]['recsen']
    for k in rs.keys():
        fileObject.write('---' + k + '--- '+ '\n')
        for r in rs[k]:
            fileObject.write(r + '\n')
            fileObject.write('-' + '\n')
        fileObject.write('---' + '\n')
        
    fileObject.write('\n\n\n' + '7. 没有和推荐句子匹配上的文章句子为：' + '\n')
    fileObject.write('------------------------' + '\n')
    for s in jmdata_map[key]['not_match_content_sen']:
        fileObject.write(s + '\n')
        fileObject.write('-' + '\n')
    
    
    fileObject.write('\n\n\n' + '没有和文章句子匹配上的推荐句子及其意图为：' + '\n')
    fileObject.write('------------------------' + '\n')
    for k,v in jmdata_map[key]['not_match_pdf_sen'].items():
        fileObject.write(k + '\t' + v + '\n')
        fileObject.write('-' + '\n')
    fileObject.close()
    i+=1

In [351]:
def getNotMatchPercent(jmdata_map, key, mode):
    labels = ['导语', '业绩变动原因', '主营业务$业务进展', '未来计划']
    rec = jmdata_map[key]['rec'+mode]
    not_match_list = list(jmdata_map[key]['not_match_pdf_' + mode].keys())
    not_match_percent = {}
    for i in labels:
        not_match_percent[i] = 0
    for k in rec.keys():
        count = 0
        for text in rec[k]:
            if text in not_match_list:
                count += 1
        not_match_percent[k] = count * 1.0 / len(rec[k])
    jmdata_map[key]['not_match_percent_' + mode] = not_match_percent
    
for key in jmdata_map.keys():
    getNotMatchPercent(jmdata_map, key, 'para')
    getNotMatchPercent(jmdata_map, key, 'sen')
    
    

In [376]:
length = len(jmdata_map.keys())
labels = ['导语', '业绩变动原因', '主营业务$业务进展', '未来计划']
cnt = {}
for l in labels:
    cnt[l] = 0
for key in jmdata_map.keys():
    for k,v in jmdata_map[key]['not_match_percent_para'].items():
        cnt[k] += v
for key in cnt:
    cnt[key] /= length
for k, v in cnt.items():
    print k, v

主营业务$业务进展 0.610144927536
导语 0.731884057971
业绩变动原因 0.878985507246
未来计划 0.797101449275


In [377]:
#-jmdata_map:
#--'season', 'year'
#---pdfurl:

#content：文章内容,                     type:list
#mapcontent：来自pdf的文章内容          type:list
#para: 该pdf的段落列表                  type:list
#sen: 该pdf的句子列表                   type:list
#paraIntent：段落对应意图               type：{'句子'：{'意图'：prob}, '句子': {'意图':prob}}
#senIntent: 句子对应意图                type: 同paraIntent
#recpara: 推荐的pdf段落                 type: {'导语': ['句子', '句子'], '未来计划': ['句子', '句子']
#recsen: 推荐的pdf句子                  type：同recpara
#not_match_content_para: 没有和推荐段落匹配上的文章句子      type：list
#not_match_content_sen: 没有和推荐句子匹配上的文章句子       type: list
#not_match_pdf_para: 没有和文章句子匹配上的pdf段落及其意图   type: {'段落':'意图'}
#not_match_pdf_sen: 没有和文章句子匹配上的pdf句子及其意图    type: {'句子': '意图'}

#not_match_para_percent: 每个意图推荐的段落中有多少比例没有用到    type: {'意图': prob}
#not_match_sen_percent： 每个意图推荐的句子中有多少比例没有用到    type: {'意图': prob}
with open('/home/zhuyuhe/mydata/jmd/percent/jmdata_map_headline.pkl', 'wb') as pickle_file:
     pickle.dump(jmdata_map, pickle_file , protocol=2)

In [ ]:
#content：文章内容,                     type:list
#mapcontent：来自pdf的文章内容          type:list
#para: 该pdf的段落列表                  type:list
#sen: 该pdf的句子列表                   type:list
#paraIntent：段落对应意图               type：{'句子'：{'意图'：prob}, '句子': {'意图':prob}}
#senIntent: 句子对应意图                type: 同paraIntent
#recpara: 推荐的pdf段落                 type: {'导语': ['句子', '句子'], '未来计划': ['句子', '句子']
#recsen: 推荐的pdf句子                  type：同recpara
#not_match_content_para: 没有和推荐段落匹配上的文章句子      type：list
#not_match_content_sen: 没有和推荐句子匹配上的文章句子       type: list
#not_match_pdf_para: 没有和文章句子匹配上的pdf段落及其意图   type: {'段落':'意图'}
#not_match_pdf_sen: 没有和文章句子匹配上的pdf句子及其意图    type: {'句子': '意图'}

#not_match_para_percent: 每个意图推荐的段落中有多少比例没有用到    type: {'意图': prob}
#not_match_sen_percent： 每个意图推荐的句子中有多少比例没有用到    type: {'意图': prob}
with open('/home/zhuyuhe/mydata/jmd/percent/jmdata_map.pkl', 'wb') as pickle_file:
     pickle.dump(jmdata_map, pickle_file , protocol=2)

In [422]:
i = 1
print len(jmdata_map[jmdata_map.keys()[i]]['mapcontent'])

print len(jmdata_map[jmdata_map.keys()[i]]['recpara'])
print len(jmdata_map[jmdata_map.keys()[i]]['recsen'])
print len(jmdata_map[jmdata_map.keys()[i]]['not_match_content_para'])
print len(jmdata_map[jmdata_map.keys()[i]]['not_match_content_sen'])
print len(jmdata_map[jmdata_map.keys()[i]]['not_match_pdf_para'])
print len(jmdata_map[jmdata_map.keys()[i]]['not_match_pdf_sen'])

print len(jmdata_map[jmdata_map.keys()[i]]['para'])
print len(jmdata_map[jmdata_map.keys()[i]]['sen'])
print jmdata_map.keys()[i]


7
4
4
3
4
5
12
20
28
http://disclosure.szse.cn/finalpage/2016-04-26/1202239317.PDF


In [457]:
#jmdata_map = pickle.load(open('/home/zhuyuhe/mydata/jmd/percent/jmd_pdf_map.pkl', 'rb'))
P_para_map = []
R_para_map = []
P_sen_map = []
R_sen_map = []
for key in jmdata_map.keys():
    print key  
    print '---------PARA-----------'
    rec = []
    for v in jmdata_map[key]['recpara'].values():
        rec.extend(v)
    rec = list(set(rec))
    P,R = a.computePR(jmdata_map[key]['mapcontent'], recList=rec)
    jmdata_map[key]['P_para'] = P
    jmdata_map[key]['R_para'] = R
    P_para_map.append(P)
    R_para_map.append(R)
    print "Pecision: " + str(P) + "    Recall: " + str(R)
    print '---------SEN-------------'
    rec = []
    for v in jmdata_map[key]['recpara'].values():
        rec.extend(v)
    rec = list(set(rec))
    P,R = a.computePR(jmdata_map[key]['mapcontent'], recList=rec)
    jmdata_map[key]['P_sen'] = P
    jmdata_map[key]['R_sen'] = R
    P_sen_map.append(P)
    R_sen_map.append(R)
    print "Pecision: " + str(P) + "    Recall: " + str(R)
    print '*******************************************************'

http://disclosure.szse.cn/finalpage/2017-04-25/1203379575.PDF
---------PARA-----------
article length: 10
recommend list length: 8
Pcount: 2
Rcount: 4
Pecision: 0.25    Recall: 0.4
---------SEN-------------
article length: 10
recommend list length: 8
Pcount: 2
Rcount: 4
Pecision: 0.25    Recall: 0.4
*******************************************************
http://www.cninfo.com.cn/finalpage/2017-04-21/1203369775.PDF
---------PARA-----------
article length: 9
recommend list length: 8
Pcount: 1
Rcount: 1
Pecision: 0.125    Recall: 0.111111111111
---------SEN-------------
article length: 9
recommend list length: 8
Pcount: 1
Rcount: 1
Pecision: 0.125    Recall: 0.111111111111
*******************************************************
http://static.sse.com.cn/disclosure/listedinfo/announcement/c/2016-08-25/600661_2016_z.pdf
---------PARA-----------
article length: 13
recommend list length: 9
Pcount: 3
Rcount: 5
Pecision: 0.333333333333    Recall: 0.384615384615
---------SEN-------------
article 

In [316]:
#结果为-1代表mapcontent长度为0
a = jmdPDFMap()
#jmdata_map = pickle.load(open('/home/zhuyuhe/mydata/jmd/percent/jmd_pdf_map.pkl', 'rb'))
url = []
for p in jmdata_map.keys():
    if jmdata_map[p]['mapcontent'].__len__() <= 5 and jmdata_map[p]:
        url.append(p)
for u in url:
    del jmdata_map[u]
P_para_map = []
R_para_map = []
P_sen_map = []
R_sen_map = []
for key in jmdata_map.keys():
    print key  
    print '---------PARA-----------'
    P,R = a.computePR(jmdata_map[key]['mapcontent'], textList=jmdata_map[key]['para'], mode = 'para')
    P_para_map.append(P)
    R_para_map.append(R)
    jmdata_map[key]['recPara'] = a.getRec()
    print "Pecision: " + str(P) + "    Recall: " + str(R)
    print '---------SEN-------------'
    P,R = a.computePR(jmdata_map[key]['mapcontent'], textList=jmdata_map[key]['sen'], mode = 'sen')
    P_sen_map.append(P)
    R_sen_map.append(R)
    jmdata_map[key]['recSen'] = a.getRec()
    print "Pecision: " + str(P) + "    Recall: " + str(R)
    print '*******************************************************'
    

http://www.cninfo.com.cn/finalpage/2017-04-19/1203340942.PDF
---------PARA-----------
article length: 9


KeyError: 'predict'

In [329]:
text = {'content':'2003年前，任教于中国科学技术大学'}
query = requests.post("https://surreal.aidigger.com/api/v1/du/finance",json=text).json()
predict = query[0]['features']['intents']
# prob = [x['prob'] for x in predict]
# i = max(prob)
# intent = predict[prob.index(i)]['name']
# print intent
for i in predict:
    print i['name'], i['prob']

意图_企业业绩_现状偏好 0.0303134600859
意图_企业业绩_现状偏坏 0.0400226827081
意图_企业业务_业务前景 0.0467662658824
意图_企业业务_业务规划 0.0436516138708
意图_企业业务_业务规模 0.0493170299364
意图_企业业务_业务介绍 0.144790515798
意图_企业活动_公司公告信息 0.032699136721
意图_企业活动_投资活动 0.0396788936686
意图_企业业绩_前景偏好 0.0299068878646
意图_企业业绩_前景偏坏 0.00347500724781
意图_企业活动_人事活动 0.00678235790352
意图_企业活动_融资活动 0.00562315851967
意图_企业基本信息介绍_企业基本信息介绍 0.0107889492896
意图_企业业绩_业绩变动原因 0.339494465053
其他 0.0958497387547
意图_评论_风险提示 0.0466737285409
意图_评论_投资建议 0.0227112940676
意图_评论_盈利预测 0.00571671782108
导语 0.00573809626573


In [458]:
print P_para_map
print R_para_map
print P_sen_map
print R_sen_map

[0.25, 0.125, 0.3333333333333333, 0.17647058823529413, 0.3333333333333333, 0.3333333333333333, 0.6, 0.09090909090909091, 0.5, 0.2222222222222222, 0.09090909090909091, 0.3, 0.1111111111111111, 0.5, 0.26666666666666666, 0.125, 0.2222222222222222, 0.4666666666666667, 0.16666666666666666]
[0.4, 0.1111111111111111, 0.38461538461538464, 0.18181818181818182, 0.5, 0.8333333333333334, 1.0, 0.13333333333333333, 1.0, 0.4444444444444444, 0.125, 0.25, 0.14285714285714285, 0.5, 0.5, 0.16666666666666666, 0.3333333333333333, 0.8, 0.2857142857142857]
[0.25, 0.125, 0.3333333333333333, 0.17647058823529413, 0.3333333333333333, 0.3333333333333333, 0.6, 0.09090909090909091, 0.5, 0.2222222222222222, 0.09090909090909091, 0.3, 0.1111111111111111, 0.5, 0.26666666666666666, 0.125, 0.2222222222222222, 0.4666666666666667, 0.16666666666666666]
[0.4, 0.1111111111111111, 0.38461538461538464, 0.18181818181818182, 0.5, 0.8333333333333334, 1.0, 0.13333333333333333, 1.0, 0.4444444444444444, 0.125, 0.25, 0.142857142857142

In [459]:
import numpy as np
print 'p_para mean: ' + str(np.mean(filter(lambda x: x!=-1, P_para_map)))
print 'r_para mean: ' + str(np.mean(filter(lambda x: x!=-1, R_para_map)))
print 'p_sen mean: ' + str(np.mean(filter(lambda x: x!=-1, P_sen_map)))
print 'r_sen mean: ' + str(np.mean(filter(lambda x: x!=-1, R_sen_map)))

p_para mean: 0.274412859243
r_para mean: 0.425906695644
p_sen mean: 0.274412859243
r_sen mean: 0.425906695644


In [153]:
u = 'http://www.cninfo.com.cn/finalpage/2017-04-27/1203451818.PDF'
print u
for i in jmdata_map[u]['mapcontent']:
    print '-----------'
    print i
print '*************************************'

print 'recommend Paragraph:'
for i in jmdata_map[u]['recPara']:
    print '-----------'
    print i
print '***************************************'

print 'recommend Sentence:'
for i in jmdata_map[u]['recSen']:
    print '------------'
    print i

http://www.cninfo.com.cn/finalpage/2017-04-27/1203451818.PDF
-----------
 成立于 2006 年的世纪明德是专业的游学服务提供商，为 3 至 18 岁青少年提供国内游学、国际游学、研学旅行服务，业务涵盖游学产品研发、推广销售、团队运作和后续接待服务等全套流程
-----------
此外，其还面向行业提供教师培训和教育论坛服务
-----------
 报告显示，世纪明德在去年新增国内常规游学线路 14 条，国际游学线路 6 条，师训产品实现了体系化和模块化，客户学校增加了 47.38%
-----------
对于营收等经营指标的增长，报告给出了 3 点原因： 市场环境向好：国家研学旅行政策不断落地，政策环境向好，市场需求大增，客户人数增长迅速； 产品线丰富：加强了国内研学、国际游学、社会实践以及教师培训产品的研发，产品可以覆盖更大客户群，可以为存量客户提供更多的产品服务内容； 客户范围不断扩大：其去年对春秋季研学旅行市场进行了重点突破，从客户参营时间上进行延展，从一年中寒暑假的游学旺季拓展为全年旺季； 从各产品线的营收来看，国内研学产品仍然是世纪明德最重要的业务，营收占比达到 72.33%，收入增长 34.36%
-----------
 世纪明德方面表示，十余年来的积累使得其已经具备了规模优势和亿元级别的销售额，以及近三十万终端客户
-----------
未来，其将利用先发优势，利用投资并购手段，整合部分小型的游学机构
-----------
在业务层面，世纪明德将重点发力国际游学、营地的开发建设以及营地教育，通过设立教育产业并购的方式对行业进行整合
-----------
未来三到五年，世纪明德希望成为中国最大的综合性游学集团
-----------
 报告称，世纪明德在 2017 年计划完成 30 万人次的销售和接待量，实现营业收入持续快速增长
*************************************
recommend Paragraph:
-----------
公司的教师培训业务主要为教育工作者提供学习交流论坛，通过了解客户对于教育培训方面的需求，
-----------
产品研发方面，公司课程研发部门研发百余种研学课程，并汇集成册，形成了“研学旅行在中国”系

In [ ]:
jmdata_mapping = pickle.load(open('/home/zhuyuhe/mydata/jmd/percent/jmd_pdf_mapping2.pkl', 'rb'))[2:]
a = jmdPDFMap()
P_para = []
R_para = []
P_sen = []
R_sen = []
i = 0
for jmap in jmdata_mapping:
    print '******************************************************'
    print 'article ' + str(i) + ' :'
    print '--------------------'
    P,R = a.computePR(jmap['content'], jmap['pdf_url'], 'para')
    P_para.append(P)
    R_para.append(R)
    print "--PARA--  Pecision: " + str(P) + "    Recall: " + str(R)
    print '--------------------'
    P,R = a.computePR(jmap['content'], jmap['pdf_url'], 'sentence')
    P_sen.append(P)
    R_sen.append(R)
    print "--SEN--  Pecision: " + str(P) + "    Recall: " + str(R)
    print '---------------------'
    print '*******************************************************'
    i += 1
#print a.getIntent('前三季度利润为0，同比下降5%')['intent']

# jmap = jmd_mapping[0]
# print a.computePR(jmap['content'], jmap['pdf_url'], 'sentence')

In [ ]:
a = jmdPDFMap()
for jmap in jmdata_mapping[14:]:
    print '******************************************************'
    print 'article ' + str(i) + ' :'
    print '--------------------'
    P,R = a.computePR(jmap['content'], jmap['pdf_url'], 'para')
    P_para.append(P)
    R_para.append(R)
    print "--PARA--  Pecision: " + str(P) + "    Recall: " + str(R)
    print '--------------------'
    P,R = a.computePR(jmap['content'], jmap['pdf_url'], 'sentence')
    P_sen.append(P)
    R_sen.append(R)
    print "--SEN--  Pecision: " + str(P) + "    Recall: " + str(R)
    print '---------------------'
    print '*******************************************************'
    i += 1

In [ ]:
print 'P_para: ' + str(P_para)
print 'R_para: ' + str(R_para)
print 'P_sen: ' + str(P_sen)
print 'R_sen: ' + str(R_sen) 
print len(P_para)

In [ ]:
i = 0
jmdata_map = {}
P_para2 = []
P_sen2 = []
R_para2 = []
R_sen2 = []
jmdata_mapping = pickle.load(open('/home/zhuyuhe/mydata/jmd/percent/jmd_pdf_mapping2.pkl', 'rb'))[2:]
a = jmdPDFMap()
for jmap in jmdata_mapping:
    key = jmap['pdf_url']
    try:
        para = a.extractPDF(jmap['pdf_url'], 0.5, 'para')
    except BaseException:
        continue
    jmdata_map[key] = {}
    content = a.get_article_sentence(jmap['content'])
    sen = a.splitByPeriod(para)
    jmdata_map[key]['content'] = content
    jmdata_map[key]['para'] = para
    jmdata_map[key]['sen'] = sen
    print '---' + key + ' done' + '----'
    i+=1


Pdfminer Parsed time: 39s
Construct Pdf Tree time: 0s
pdflist length: 182
---http://www.cninfo.com.cn/finalpage/2017-06-20/1203635303.PDF done----
Download pdf time: 7s
download http status code: 200
Pdfminer Parsed time: 36s
Construct Pdf Tree time: 0s
pdflist length: 166
---http://disclosure.szse.cn/finalpage/2017-04-11/1203274908.PDF done----
Download pdf time: 2s
download http status code: 200
Pdfminer Parsed time: 17s
Construct Pdf Tree time: 0s
pdflist length: 46
---http://www.cninfo.com.cn/finalpage/2017-07-26/1203737887.PDF done----
Download pdf time: 6s
download http status code: 200
Pdfminer Parsed time: 35s
Construct Pdf Tree time: 0s
pdflist length: 86
---http://disclosure.szse.cn/finalpage/2017-08-25/1203872097.PDF done----
Download pdf time: 1s
download http status code: 200
Pdfminer Parsed time: 6s
Construct Pdf Tree time: 0s
pdflist length: 11
---http://disclosure.szse.cn/finalpage/2016-10-18/1202765393.PDF done----
Download pdf time: 1s
download http status code: 200
P

In [399]:
#4篇季报，19篇年报
seasonArticle = ['http://disclosure.szse.cn/finalpage/2016-10-25/1202780794.PDF', 'http://disclosure.szse.cn/finalpage/2016-04-26/1202239317.PDF', 'http://disclosure.szse.cn/finalpage/2016-10-27/1202793355.PDF','http://disclosure.szse.cn/finalpage/2017-04-26/1203386046.PDF']
yearlyArticle = [u'http://www.cninfo.com.cn/finalpage/2017-04-19/1203340942.PDF', 
                 u'http://disclosure.szse.cn/finalpage/2017-04-21/1203344261.PDF',
                 u'http://static.sse.com.cn/disclosure/listedinfo/announcement/c/2017-04-15/601801_2016_n.pdf', 
                 u'http://disclosure.szse.cn/finalpage/2017-07-28/1203739610.PDF', 
                 u'http://www.cninfo.com.cn/finalpage/2017-06-20/1203635303.PDF', 
                 u'http://disclosure.szse.cn/finalpage/2017-08-10/1203778984.PDF', 
                 u'http://disclosure.szse.cn/finalpage/2017-03-28/1203210341.PDF', 
                 u'http://static.sse.com.cn/disclosure/listedinfo/announcement/c/2016-08-25/600661_2016_z.pdf', 
                 u'http://www.cninfo.com.cn/finalpage/2017-07-28/1203751786.PDF', 
                 u'http://disclosure.szse.cn/finalpage/2017-04-25/1203379575.PDF', 
                 u'http://www.cninfo.com.cn/finalpage/2017-05-10/1203508904.PDF',
                 u'http://disclosure.szse.cn/finalpage/2017-04-24/1203360123.PDF',
                 u'http://www.cninfo.com.cn/finalpage/2017-04-27/1203451818.PDF', 
                 u'http://disclosure.szse.cn/finalpage/2017-03-18/1203174359.PDF',
                 u'http://www.cninfo.com.cn/finalpage/2017-04-19/1203341493.PDF', 
                 u'http://www.cninfo.com.cn/finalpage/2017-04-21/1203369775.PDF',
                 u'http://www.cninfo.com.cn/finalpage/2017-07-26/1203737887.PDF',
                 u'http://disclosure.szse.cn/finalpage/2017-03-21/1203181705.PDF',
                 u'http://disclosure.szse.cn/finalpage/2017-04-11/1203274908.PDF']


4
23
19


In [404]:
print yearlyArticle

[u'http://www.cninfo.com.cn/finalpage/2017-04-19/1203340942.PDF', u'http://disclosure.szse.cn/finalpage/2017-04-21/1203344261.PDF', u'http://static.sse.com.cn/disclosure/listedinfo/announcement/c/2017-04-15/601801_2016_n.pdf', u'http://disclosure.szse.cn/finalpage/2017-07-28/1203739610.PDF', u'http://www.cninfo.com.cn/finalpage/2017-06-20/1203635303.PDF', u'http://disclosure.szse.cn/finalpage/2017-08-10/1203778984.PDF', u'http://disclosure.szse.cn/finalpage/2017-03-28/1203210341.PDF', u'http://static.sse.com.cn/disclosure/listedinfo/announcement/c/2016-08-25/600661_2016_z.pdf', u'http://www.cninfo.com.cn/finalpage/2017-07-28/1203751786.PDF', u'http://disclosure.szse.cn/finalpage/2017-04-25/1203379575.PDF', u'http://www.cninfo.com.cn/finalpage/2017-05-10/1203508904.PDF', u'http://disclosure.szse.cn/finalpage/2017-04-24/1203360123.PDF', u'http://www.cninfo.com.cn/finalpage/2017-04-27/1203451818.PDF', u'http://disclosure.szse.cn/finalpage/2017-03-18/1203174359.PDF', u'http://www.cninfo.co

In [403]:
for i in jmdata_map['http://disclosure.szse.cn/finalpage/2016-10-25/1202780794.PDF']['content']:
    print i
    print '-'

今日傍晚， 方直科技 （300235）发布了2016年第三季度报告，报告显示，2016年第三季度，其营业总收入为3610.01万元，归属于上市公司股东的净利润为1331.05万元
-
 营收数据 统计数据显示，截至报告期末，方直科技总资产为3.80亿元
-
其第三季度营业总收入为3610.01万元，同比下降14.40%；归属于上市公司股东的净利润为1331.05万元，同比增加5.20%
-
 2016年前三季度，方直科技营业总收入为7936.14万元，同比下降2.89%；归属于上市公司股东的净利润为1817.50万元，同比下降22.32% 据悉，本报告期末应收账款比年初增长108.12%，主要原因是秋季教育系统征订业务确认收入形成的应收账款较大所致
-
 业务展望 前三季度，方直科技核心产品业务及其结构未发生重大变化，金太阳教育软件的开发、销售业务仍是公司的核心业务
-
华南、华北和华东三个地区是产品销售比较集中的区域，报告期内各地区销售收入占主营业务收入比例分别为67.08%、17.92%、8.6%
-
 未来，在产品研发方面，方直科技将继续巩固强化公司在内容资源方面的优势，建立和完善语文、英语、数学等多学科、多版本的同步教学优质资源开发
-
不断丰富金太阳产品集群多样性，加快推进各应用端全覆盖，促使更多的免费使用用户转化为购买用户
-
 另一方面，方直科技将不断加大对网络技术开发、学习资源开发及网络产品运营人力的投入，以加速向互联网企业转型
-
 风险应对 目前方直科技的销售区域仍主要以珠三角、长三角及京津地区等大中城市为主，市场区域相对集中并存在一定风险，可能对其未来盈利能力造成影响
-
 方直科技将通过技术研发和产品拓展计划、营销渠道拓展计划解决上述产品及销售区域较为集中的风险
-
 如果方直科技对行业关键技术的发展动态不能及时掌控，对教育教学当中的新理念以及广大教师和学生的教学需求不能正确把握，研发的产品将不具备竞争力
-
 因此，方直科技将积极投入同步教学软件及在线教育网络服务开发平台的开发，加强基础教育项目研究
-
 来源：                                    芥末堆
-


In [465]:
yearlyArticle[0]

u'http://www.cninfo.com.cn/finalpage/2017-04-19/1203340942.PDF'

In [469]:
#年报规则探索
for u in yearlyArticle[:2]:
    for i in jmdata_map[u]['mapcontent']:
        print i
        print '-'
    print '--------------------------'
    rec = []
    for v in jmdata_map[u]['recpara'].values():
        rec.extend(v)
    for r in rec:
        print r
        print '--'
    
    print '***********************************'

 艾的教育成立于 2009 年，主要为 0-12 岁婴幼儿提供个性化成长和教育综合解决方案，其客户包括早教中心、幼儿园和培训机构，艾的教育提供系统的个性化教育解决方案、培训服务和运营支持
-
2016 年 1 月，艾的教育在新三板挂牌
-
 基于之前积累的数万家幼儿园作为目标客户资源，艾的教育通过招商合作等多种模式进行新型销售渠道的搭建，包括招募全国地市代理商，发展面向园所和机构的直销运营团队，与第三方园所或机构服务商合作推广销售等
-
 艾的教育的主要收入来源于基于移动互联网和大数据的个性化幼教产品和解决方案，包括综合评量、个性化方案以及相关教育专家的在线课堂和咨询，同时支持园所或培训机构构建爱宝儿童个性化成长评量馆
-
年报显示，这部分已超过公司年度收入的 55%，未来个性化产品和解决方案将在公司收入中占据更大比例，此外公司的收入还包括幼儿园品牌等
-
 对于利润的下降，报告给出的原因是由于持续加大个性化科技教育产品的研发投入，在丰富评量产品的同时，对原有产品进行了迭代升级，并进一步加大投入市场渠道推广所致
-
 在运营层面，通过线上线下运营从 B 端衍生到 C 端
-
线下运营层面，以园长会和家长会为主，全年举办 900 多场，覆盖 300 多个地市和县区，直接影响近 1 万园长和 20 万家长；线上运营层面，艾的教育构建了园长付费社群“园长帮”，付费会员近 5000 人
-
面向家长的爱宝个性化教育平台“最爱宝”，家长粉丝近 60 万人
-
--------------------------
公司主营业务为面向全国的早教中心、幼儿园和培训机构，提供以儿童成长大数据为基础的个性化教
育解决方案，涵盖产品研发、推广销售、培训服务、后期支撑、数据运营等教育业务全流程。具体包括轻、
中、重三种模式的解决方案：轻量级的个性化解决方案包括面向 B 端以 SaaS 模式（Software as a Service）
提供服务的儿童成长教育相关的各种评量；中量级的个性化方案包括综合评量、个性化方案以及相关教育
专家的在线课堂和咨询，同时支持园所或培训机构构建爱宝儿童个性化成长评量馆；重量级的个性化方案，
是在中量级方案的基础上，既进行了产品种类的拓展，如从儿童综合评量衍生到儿童专注力、体智能、家
庭教育测评等多个方面，又向 B 端提供深度的线上、线下课程，还向 C 端提供

In [405]:
#季报规则探索
for u in seasonArticle:
    for i in jmdata_map[u]['mapcontent']:
        print i
        print '-'
    print '***********'

 业务展望 前三季度，方直科技核心产品业务及其结构未发生重大变化，金太阳教育软件的开发、销售业务仍是公司的核心业务
-
华南、华北和华东三个地区是产品销售比较集中的区域，报告期内各地区销售收入占主营业务收入比例分别为67.08%、17.92%、8.6%
-
 未来，在产品研发方面，方直科技将继续巩固强化公司在内容资源方面的优势，建立和完善语文、英语、数学等多学科、多版本的同步教学优质资源开发
-
不断丰富金太阳产品集群多样性，加快推进各应用端全覆盖，促使更多的免费使用用户转化为购买用户
-
 另一方面，方直科技将不断加大对网络技术开发、学习资源开发及网络产品运营人力的投入，以加速向互联网企业转型
-
 风险应对 目前方直科技的销售区域仍主要以珠三角、长三角及京津地区等大中城市为主，市场区域相对集中并存在一定风险，可能对其未来盈利能力造成影响
-
 方直科技将通过技术研发和产品拓展计划、营销渠道拓展计划解决上述产品及销售区域较为集中的风险
-
 如果方直科技对行业关键技术的发展动态不能及时掌控，对教育教学当中的新理念以及广大教师和学生的教学需求不能正确把握，研发的产品将不具备竞争力
-
 因此，方直科技将积极投入同步教学软件及在线教育网络服务开发平台的开发，加强基础教育项目研究
-
***********
 以 EdSaas 业务、继续教育业务、家校互动升级业务，以及学科升学业务四大业务群为核心，全通教育的业务均在稳步推进，截至报告期末： EdSaas 方面，作为全课云的入口，移动校园门户累计注册学校数达 16357 所，教师及家长关注数达 372 万人
-
 继续教育业务方面，“国培计划”项目陆续启动, 在巩固扩大传统优势培训项目的同时，全通继教拓展了区域性整合研修项目，针对地市县等区域性培训新需求自主进行课程研发及产品设计，取得了初步进展
-
 家校互动升级业务方面： 成长帮手产品内容不断丰富并持续通过互联网运营增强用户活跃，已在广东、江西、浙江、河北、贵州、重庆等多个省份与基础运营商平台深度合作，加强互联网运营及线上推广，并推出“教育大咖直播课”、“专家答疑”等模块
-
 学科辅导产品全课通迭代升级，尝试在多个运营商平台深度运营，提升用户粘性，2017 年初上线的“全课答疑”不断改进答题技术及服务，截至报告期末使用用户超过 35 万； 动力加一卡通产品运营型业务

In [389]:
#年报规则探索
urls = ['http://www.cninfo.com.cn/finalpage/2017-04-21/1203369775.PDF',
 'http://www.cninfo.com.cn/finalpage/2017-07-26/1203737887.PDF',
 'http://disclosure.szse.cn/finalpage/2017-03-21/1203181705.PDF',
 'http://disclosure.szse.cn/finalpage/2017-04-11/1203274908.PDF']
for u in urls:
    for i in jmdata_map[u]['content']:
        print i
        print '-'
    print '***********'
    
    
'''第一节声明与提示
第二节公司概况
第三节会计数据和财务指标摘要
第四节管理层讨论与分析
第五节重要事项
第六节股本变动及股东情况
第七节融资及分配情况
第八节董事、监事、高级管理人员及员工情况
第九节公司治理及内部控制
第十节财务报告'''



芥末堆 4 月 24 讯，正保远程的国内控股子公司正保育才日前公布了其 2016 年年度报告，去年其实现营收 3518.96 万元，同比增长 36.71%；归属于挂牌公司股东的净利润 952.96 万元，同比增长 5.23%
-
 正保育才是创业培训与服务解决方案供应商，为学员提供创业培训方案，并提供包括创业活动、项目资源对接及公共服务对接等在内的多项创业服务
-
目前，其营业收入主要来自于创业培训
-
 正保育才的销售客户主要为各地的就业指导中心、职业培训中心（或其委托机构）等负责组织实施本地区就业培训的单位以及各地的职业培训学校等，并不直接面向学员收取培训费用
-
正保育才主要负责创业培训项目的组织实施，包括提供培训课程的内容与技术支持、组织配置授 课培训师以及学员的招生等
-
 由于国家和地方政府陆续出台了对于创业培训费用补贴的相关规定，学员一般可免费参加相关创业培训项目
-
培训结束后，就业指导中心、职业培训中心、各地职业培训学校等单位将根据当地的鼓励政策文件，按要求向当地政府申请创业培训补贴， 正保育才按照双方合同约定，确认相应的培训业务收入
-
 据年度报告显示，正保育才去年培训学员 12.6万人，共计开班 3344 个，同比增加 46.8%
-
合作培训的省市 25 个，增加了与云南、湖南、湖北三省的合作，在这三个省分别与省就业局、培训机构和高校的创业培训或实训平台的合作
-
 去年，正保育才进行了两次股权融资：2016 年 1 月增发股份给北京正保同心管理咨询合伙企业（有限合伙），筹资 31,734,000 元；2016 年 12 月增发股份给朱正东、胡联奎和南京正保通信网络技术有限公司，筹资83,341,200 元
-
 目前，正保育才正在与重庆、福建的相关机构、高校洽谈合作
-
报告称，今年与其合作的省市预计将达 27-28个
-
 来源：                                    芥末堆
-
***********
芥末堆 7 月 28 日讯，近日，山东朗朗教育发布了 2017 年半年度报告，报告显示，截至 2017 年 6 月 30 日，公司资产总额 1.43 亿元，较期初增长 30.17%
-
 朗朗教育是一家集产品研发、发行销售、服务跟踪、培训创新、智慧教育于 一体的学前教育综合解决方案提供商，专注于学前教育资

'\xe7\xac\xac\xe4\xb8\x80\xe8\x8a\x82\xe5\xa3\xb0\xe6\x98\x8e\xe4\xb8\x8e\xe6\x8f\x90\xe7\xa4\xba\n\xe7\xac\xac\xe4\xba\x8c\xe8\x8a\x82\xe5\x85\xac\xe5\x8f\xb8\xe6\xa6\x82\xe5\x86\xb5\n\xe7\xac\xac\xe4\xb8\x89\xe8\x8a\x82\xe4\xbc\x9a\xe8\xae\xa1\xe6\x95\xb0\xe6\x8d\xae\xe5\x92\x8c\xe8\xb4\xa2\xe5\x8a\xa1\xe6\x8c\x87\xe6\xa0\x87\xe6\x91\x98\xe8\xa6\x81\n\xe7\xac\xac\xe5\x9b\x9b\xe8\x8a\x82\xe7\xae\xa1\xe7\x90\x86\xe5\xb1\x82\xe8\xae\xa8\xe8\xae\xba\xe4\xb8\x8e\xe5\x88\x86\xe6\x9e\x90\n\xe7\xac\xac\xe4\xba\x94\xe8\x8a\x82\xe9\x87\x8d\xe8\xa6\x81\xe4\xba\x8b\xe9\xa1\xb9\n\xe7\xac\xac\xe5\x85\xad\xe8\x8a\x82\xe8\x82\xa1\xe6\x9c\xac\xe5\x8f\x98\xe5\x8a\xa8\xe5\x8f\x8a\xe8\x82\xa1\xe4\xb8\x9c\xe6\x83\x85\xe5\x86\xb5\n\xe7\xac\xac\xe4\xb8\x83\xe8\x8a\x82\xe8\x9e\x8d\xe8\xb5\x84\xe5\x8f\x8a\xe5\x88\x86\xe9\x85\x8d\xe6\x83\x85\xe5\x86\xb5\n\xe7\xac\xac\xe5\x85\xab\xe8\x8a\x82\xe8\x91\xa3\xe4\xba\x8b\xe3\x80\x81\xe7\x9b\x91\xe4\xba\x8b\xe3\x80\x81\xe9\xab\x98\xe7\xba\xa7\xe7\xae\xa1\xe7\x90\x86\